In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import glob

import pandas as pd

In [9]:
## delete the folders not in live config ( extra crap )
live_config_files = glob.glob('cfgs_live/*', recursive=True)
allowed_symbols = []
for f in live_config_files:
    symbol = f.split('\\')[-1]
    symbol = symbol.replace('.json', 'USDT')
    allowed_symbols.append(symbol)

allowed_symbols = ["ALICEUSDT","XRPUSDT","HOTUSDT"]

In [10]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_308204/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_308204/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_308204/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_308204/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs
0,\1000XECUSDT\plots\2022-03-15T184506\backtest_...,5.5,1000XECUSDT,True,13.140,0.06520,100.000,0.9340,True,29.910,0.1540,100.0,0.9526
1,\1000XECUSDT\plots\2022-03-16T132802\backtest_...,5.5,1000XECUSDT,True,13.410,0.06740,100.000,0.9340,True,30.410,0.1440,100.0,0.9526
2,\1INCHUSDT\plots\2022-03-16T133853\backtest_re...,5.5,1INCHUSDT,True,2.589,0.00706,100.000,0.8369,True,26.760,0.0778,100.0,0.7473
3,\AAVEUSDT\plots\2022-03-14T073611\backtest_res...,5.5,AAVEUSDT,True,-11.590,-0.05940,100.000,0.7649,True,54.850,0.2140,100.0,0.7848
4,\ADAUSDT\plots\2022-03-16T140203\backtest_resu...,5.5,ADAUSDT,True,15.100,0.04030,100.000,0.9334,True,9.646,0.0328,100.0,0.8184
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,\XRPUSDT\plots\2022-03-17T012439\backtest_resu...,5.5,XRPUSDT,True,-59.890,-1.44000,5.992,0.6186,True,15.470,0.0497,100.0,0.8121
110,\XRPUSDT\plots\2022-03-17T163011\backtest_resu...,5.5,XRPUSDT,True,5.994,0.01830,100.000,0.9675,True,8.546,0.0299,100.0,0.8323
111,\ZENUSDT\plots\2022-03-17T013136\backtest_resu...,5.5,ZENUSDT,True,13.990,0.03850,100.000,0.8480,True,31.140,0.0897,100.0,0.8627
112,\ZILUSDT\plots\2022-03-17T013628\backtest_resu...,5.5,ZILUSDT,True,-9.182,-0.03070,100.000,0.8849,True,17.220,0.0519,100.0,0.8273


Now filter the dataframe

In [11]:
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['short'] == True)
              # & (big_data['adgt_%'] > 0.1)
              # & (big_data['bnkrl_%'] > 95.0)
              # & (big_data['bnkrs_%'] > 95.0)
              # & (big_data['ebrs'] > .85)
              # & (big_data['ebrl'] > .85)
              & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs,adgt_%
8,\ALICEUSDT\plots\2022-03-16T142312\backtest_re...,5.5,ALICEUSDT,True,14.600,0.0296,100.000,0.9190,True,-92.140,-1.4400,5.90,0.06565,-1.4104
9,\ALICEUSDT\plots\2022-03-17T160925\backtest_re...,5.5,ALICEUSDT,True,7.837,0.0222,100.000,0.9479,True,10.670,0.1320,40.37,0.33100,0.1542
55,\HOTUSDT\plots\2022-03-16T195049\backtest_resu...,5.5,HOTUSDT,True,-58.730,-1.4400,5.997,0.4669,True,16.930,0.0542,100.00,0.68860,-1.3858
56,\HOTUSDT\plots\2022-03-17T161609\backtest_resu...,5.5,HOTUSDT,True,-4.725,-0.0155,100.000,0.9328,True,7.711,0.0329,100.00,0.73260,0.0174
109,\XRPUSDT\plots\2022-03-17T012439\backtest_resu...,5.5,XRPUSDT,True,-59.890,-1.4400,5.992,0.6186,True,15.470,0.0497,100.00,0.81210,-1.3903
110,\XRPUSDT\plots\2022-03-17T163011\backtest_resu...,5.5,XRPUSDT,True,5.994,0.0183,100.000,0.9675,True,8.546,0.0299,100.00,0.83230,0.0482


In [12]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_308204/2043545472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
8,\ALICEUSDT\plots\2022-03-16T142312\backtest_re...,ALICEUSDT,-1.4104,0.0296,-1.4400,100.000,5.90,0.9190,0.06565
9,\ALICEUSDT\plots\2022-03-17T160925\backtest_re...,ALICEUSDT,0.1542,0.0222,0.1320,100.000,40.37,0.9479,0.33100
55,\HOTUSDT\plots\2022-03-16T195049\backtest_resu...,HOTUSDT,-1.3858,-1.4400,0.0542,5.997,100.00,0.4669,0.68860
56,\HOTUSDT\plots\2022-03-17T161609\backtest_resu...,HOTUSDT,0.0174,-0.0155,0.0329,100.000,100.00,0.9328,0.73260
109,\XRPUSDT\plots\2022-03-17T012439\backtest_resu...,XRPUSDT,-1.3903,-1.4400,0.0497,5.992,100.00,0.6186,0.81210
110,\XRPUSDT\plots\2022-03-17T163011\backtest_resu...,XRPUSDT,0.0482,0.0183,0.0299,100.000,100.00,0.9675,0.83230
